In [ ]:
from flytekit.configuration import set_flyte_config_file, platform
set_flyte_config_file("notebook.config")
print("Connected to {}".format(platform.URL.get()))

from jupyter_helpers import print_console_url

# Execute registered Tasks

## Option 1: Execute without having the right environment setup
This option involves fetching the remote task and then requesting an execution in your own namespace.
This concept is what is also used in Sharing.

To start an execution following pieces of information are required
 - project: The project where the task was registered
 - domain: Domain where the task was registered
 - name: Name of the registered task. This is the canonical name (usually includes the module name). You can lookup using flyte-cli or console or arrive at it using "root_module.module....task" pattern
 - Version! This is the version of the registered task
 
**Note** Version is the most important to pinpoint the task to execute. At the same time Flyte can have multiple versions active. Cookbook uses Git-SHA to version a release. So you can use the latest git-sha from flytesnacks as the release identifier.


In [ ]:
from flytekit.common.tasks.task import SdkTask
version = "ade80023b74f9810fe720471b8926d6b991fc879" ## Replace this
remote_task = SdkTask.fetch(project="flytesnacks", domain="development", name="recipes.interaction.interaction.scale", version=version)

In [ ]:
exe = remote_task.execute(project="flytesnacks", domain="development", inputs={"image":"https://miro.medium.com/max/1400/1*qL8UYfaStcEo_YVPrA4cbA.png"})
print_console_url(exe)

### Load outputs for introspection
**TODO** This does not work on sandbox environment today as minio is not available on the same port

In [ ]:
exe.sync()
# In sandbox outputs does not work because the dns points to k8s local address, but this is not available on localhost
#. we can create a hostfile entry but that would be too complicated
exe.outputs

## Option 2: Execute by loading the current code 

This is only supported for non-container (like queries etc), pre-built containers like Sagemaker etc and Raw containers tasks. For tasks that are written in containers, this is not directly supported as it would easily confuse the user about false iteration, without building a container.

**NOTE** this is an active area of work, come back again and you will see an update.
